## 0: The Set-Up

In [1]:
# import all the things
import pandas as pd
import numpy as np
from tqdm import tqdm
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
print('imported')

imported


In [2]:
# read in data exported from Sirsi and cleaned in OpenRefine
sirsi = pd.read_csv('sirsi-issn-data.csv', index_col='035$a')
sirsi.head()

,type,title-proper,020,022
035$a,,,,
(Sirsi) 1545942,issn,Nature biotechnology,NaN,1087-0156
(Sirsi) 1546425,issn,Journal of public policy,NaN,0143-814X
(Sirsi) 1601451,issn,Journal of membrane science,NaN,0376-7388
(Sirsi) 1668844,issn,Journal of pathology,NaN,1096-9896||0022-3417
(Sirsi) 1669264,issn,Reviews in medical virology,NaN,1099-1654||1052-9276


In [3]:
# filter sirsi data to show all rows where no ISBNs exist and where the 'type' column is ISSN
filtered_issns = sirsi.loc[(sirsi['020'].isna()) & (sirsi.type.eq('issn'))]
filtered_issns.head()

,type,title-proper,020,022
035$a,,,,
(Sirsi) 1545942,issn,Nature biotechnology,NaN,1087-0156
(Sirsi) 1546425,issn,Journal of public policy,NaN,0143-814X
(Sirsi) 1601451,issn,Journal of membrane science,NaN,0376-7388
(Sirsi) 1668844,issn,Journal of pathology,NaN,1096-9896||0022-3417
(Sirsi) 1669264,issn,Reviews in medical virology,NaN,1099-1654||1052-9276


In [4]:
# export the Sirsi record numbers and ISSNs to separate lists
sirsi_record_numbers = filtered_issns.index.tolist()
sirsi_issns = filtered_issns['022'].tolist()

In [5]:
# read in data from SFX for JSTOR records, cleaned in OpenRefine
jstor = pd.read_csv('JSTOR-1_condensed_deduped_cleaned.csv', index_col='obj_id')
jstor.head()

,pissn,eissn,TITLE,TITLE_MAIN
obj_id,,,,
991042730655002,0001-026X,2326-9014,AAUP Bulletin,AAUP Bulletin
110992357244018,0001-2343,NaN,ARSP: Archiv für Rechts- und Sozialphilosophie...,ARSP: Archiv für Rechts- und Sozialphilosophie...
110985822449210,0001-2890,2168-8052,AV communication review : audio visual communi...,AV communication review : audio visual communi...
954921333007,0001-4273,1948-0989,Academy of Management Journal,Academy of Management Journal
954921333014,0001-4826,1558-7967,Accounting Review,Accounting Review


In [6]:
# drop the TITLE_MAIN column, we won't need it
jstor = jstor.drop(['TITLE_MAIN'], axis=1)
jstor.head()

,pissn,eissn,TITLE
obj_id,,,
991042730655002,0001-026X,2326-9014,AAUP Bulletin
110992357244018,0001-2343,NaN,ARSP: Archiv für Rechts- und Sozialphilosophie...
110985822449210,0001-2890,2168-8052,AV communication review : audio visual communi...
954921333007,0001-4273,1948-0989,Academy of Management Journal
954921333014,0001-4826,1558-7967,Accounting Review


In [7]:
# isolate e-ISSN column & drop NaN values
jstor_eissns = jstor[['eissn']].dropna()
jstor_eissns.head()

,eissn
obj_id,
991042730655002,2326-9014
110985822449210,2168-8052
954921333007,1948-0989
954921333014,1558-7967
110978976471213,2064-7182


In [8]:
# isolate print ISSN column & drop NaN values
jstor_pissns = jstor[['pissn']].dropna()
jstor_pissns.head()

,pissn
obj_id,
991042730655002,0001-026X
110992357244018,0001-2343
110985822449210,0001-2890
954921333007,0001-4273
954921333014,0001-4826


In [9]:
# export the JSTOR IDs and ISSNs to separate lists
jstor_eissn_record_numbers = jstor_eissns.index.tolist()
jstor_eissns = jstor_eissns.eissn.tolist()

jstor_pissn_record_numbers = jstor_pissns.index.tolist()
jstor_pissns = jstor_pissns.pissn.tolist()

Here's where the fun begins...
### 1: ISSN Matching

In [10]:
# create empty lists for matching ISSNs and IDs
jstor_matches_e = []
sirsi_matches_e = []
jstor_matches_p = []
sirsi_matches_p = []

In [11]:
# the meat: go through e-ISSN and p-ISSN lists and match to Sirsi
# records; save them to the above empty lists

for entry in tqdm(jstor_eissns):
    matcher = [s for s in sirsi_issns if entry in s]
    if matcher:
        for match in matcher:
            match_index = sirsi_issns.index(match)
            sirsi_match = sirsi_record_numbers[match_index]
            jstor_match_index = jstor_eissns.index(entry)
            jstor_match = jstor_eissn_record_numbers[jstor_match_index]
            jstor_matches_e.append(jstor_match)
            sirsi_matches_e.append(sirsi_match)
    else:
        pass

for entry in tqdm(jstor_pissns):
    matcher = [s for s in sirsi_issns if entry in s]
    if matcher:
        for match in matcher:
            match_index = sirsi_issns.index(match)
            sirsi_match = sirsi_record_numbers[match_index]
            jstor_match_index = jstor_pissns.index(entry)
            jstor_match = jstor_pissn_record_numbers[jstor_match_index]
            jstor_matches_p.append(jstor_match)
            sirsi_matches_p.append(sirsi_match)
    else:
        pass

100%|████████████████████████████████████| 3741/3741 [00:02<00:00, 1249.70it/s]


In [12]:
# now zip up the results into two dataframes
jstor_dfe = pd.DataFrame()
jstor_dfe['sfx_ID'] = jstor_matches_e
jstor_dfe['sirsi_ID'] = sirsi_matches_e
jstor_dfe.head()

,sfx_ID,sirsi_ID
0,954921333007,(Sirsi) 1652669
1,110978976471219,(Sirsi) 2919565
2,954925374832,(Sirsi) 2529785
3,954921333017,(Sirsi) 1717568
4,954925375839,(Sirsi) 1543554


In [13]:
jstor_dfp = pd.DataFrame()
jstor_dfp['sfx_ID'] = jstor_matches_p
jstor_dfp['sirsi_ID'] = sirsi_matches_p
jstor_dfp.head()

,sfx_ID,sirsi_ID
0,110992357244018,(Sirsi) 2654386
1,110985822449210,(Sirsi) 2233714
2,954921333007,(Sirsi) 1652669
3,954921333014,(Sirsi) 1652671
4,110978976471213,(Sirsi) 2915767


## 2: Clean-up

In [14]:
# are there multiple Sirsi IDs for SFX IDs?

dupe1 = jstor_dfe['sfx_ID'].duplicated()
dupe1.value_counts(normalize=True).apply(lambda x: x * 100)

False    98.303835
True      1.696165
Name: sfx_ID, dtype: float64

In [15]:
dupe2 = jstor_dfp['sfx_ID'].duplicated()
dupe2.value_counts(normalize=True).apply(lambda x: x * 100)

False    95.856821
True      4.143179
Name: sfx_ID, dtype: float64

In [16]:
# Yep, we better zip them up into non-duplicate key columns

jstor_dfe = jstor_dfe.set_index('sfx_ID')
jstor_dfe = jstor_dfe.groupby('sfx_ID').agg(lambda x: '||'.join(x))
jstor_dfe.head()

,sirsi_ID
sfx_ID,
954921333007,(Sirsi) 1652669
954921333017,(Sirsi) 1717568
954921334025,(Sirsi) 2529784
954921334032,(Sirsi) 1893654
954921334036,(Sirsi) 1775290


In [17]:
jstor_dfp = jstor_dfp.set_index('sfx_ID')
jstor_dfp = jstor_dfp.groupby('sfx_ID').agg(lambda x: '||'.join(x))
jstor_dfp.head()

,sirsi_ID
sfx_ID,
954921333007,(Sirsi) 1652669
954921333014,(Sirsi) 1652671
954921333016,(Sirsi) 1652672
954921333017,(Sirsi) 1717568
954921334025,(Sirsi) 2529784


In [18]:
# Merge the two into one frame
final_jstor_df = pd.merge(jstor_dfe, jstor_dfp, on='sfx_ID', how='outer')
final_jstor_df

,sirsi_ID_x,sirsi_ID_y
sfx_ID,,
954921333007,(Sirsi) 1652669,(Sirsi) 1652669
954921333017,(Sirsi) 1717568,(Sirsi) 1717568
954921334025,(Sirsi) 2529784,(Sirsi) 2529784
954921334032,(Sirsi) 1893654,(Sirsi) 1893654
954921334036,(Sirsi) 1775290,(Sirsi) 1775290
954921337086,(Sirsi) 2471107,(Sirsi) 2471107
954921338099,(Sirsi) 1627265,(Sirsi) 1627265
954921341146,(Sirsi) 1666028,(Sirsi) 1666028
954921341150,(Sirsi) 1627267,(Sirsi) 1627267


In [19]:
# Let's fill the NaN values with empty spaces
final_jstor_df['sirsi_ID_y'].fillna('', inplace=True)
final_jstor_df['sirsi_ID_x'].fillna('', inplace=True)
final_jstor_df

,sirsi_ID_x,sirsi_ID_y
sfx_ID,,
954921333007,(Sirsi) 1652669,(Sirsi) 1652669
954921333017,(Sirsi) 1717568,(Sirsi) 1717568
954921334025,(Sirsi) 2529784,(Sirsi) 2529784
954921334032,(Sirsi) 1893654,(Sirsi) 1893654
954921334036,(Sirsi) 1775290,(Sirsi) 1775290
954921337086,(Sirsi) 2471107,(Sirsi) 2471107
954921338099,(Sirsi) 1627265,(Sirsi) 1627265
954921341146,(Sirsi) 1666028,(Sirsi) 1666028
954921341150,(Sirsi) 1627267,(Sirsi) 1627267


In [20]:
# Now concatenate the Sirsi columns into one and get rid of the old ones
final_jstor_df['sirsi_ID'] = final_jstor_df['sirsi_ID_x']+ '||' + final_jstor_df['sirsi_ID_y']
final_jstor_df = final_jstor_df.drop(['sirsi_ID_x', 'sirsi_ID_y'], axis=1)
final_jstor_df

,sirsi_ID
sfx_ID,
954921333007,(Sirsi) 1652669||(Sirsi) 1652669
954921333017,(Sirsi) 1717568||(Sirsi) 1717568
954921334025,(Sirsi) 2529784||(Sirsi) 2529784
954921334032,(Sirsi) 1893654||(Sirsi) 1893654
954921334036,(Sirsi) 1775290||(Sirsi) 1775290
954921337086,(Sirsi) 2471107||(Sirsi) 2471107
954921338099,(Sirsi) 1627265||(Sirsi) 1627265
954921341146,(Sirsi) 1666028||(Sirsi) 1666028
954921341150,(Sirsi) 1627267||(Sirsi) 1627267


In [21]:
# Now finish cleaning up the Sirsi record IDs by keeping only unique
# values and getting rid of errant delimitors
final_jstor_df['sirsi_ID'] = final_jstor_df['sirsi_ID'].apply(lambda x: '||'.join(np.unique(x.split('||'))))
final_jstor_df['sirsi_ID'] = final_jstor_df['sirsi_ID'].str.replace('^\|\|', '')
final_jstor_df

,sirsi_ID
sfx_ID,
954921333007,(Sirsi) 1652669
954921333017,(Sirsi) 1717568
954921334025,(Sirsi) 2529784
954921334032,(Sirsi) 1893654
954921334036,(Sirsi) 1775290
954921337086,(Sirsi) 2471107
954921338099,(Sirsi) 1627265
954921341146,(Sirsi) 1666028
954921341150,(Sirsi) 1627267


### 3: Fuzzy Title Matches

In [22]:
# export the SFX record IDs from the original (master) list and the cleaned CSV
jstor_master_index = jstor.index.tolist()
jstor_matches_cleaned_index = final_jstor_df.index.tolist()

# remove the cleaned CSV SFX record IDs from the master list, leaving the unmatched SFX JSTOR records
for record in jstor_matches_cleaned_index:
    jstor_master_index.remove(record)

len(jstor_master_index)

278

In [23]:
leftovers = jstor.loc[jstor_master_index]
leftovers

,pissn,eissn,TITLE
obj_id,,,
991042730655002,0001-026X,2326-9014,AAUP Bulletin
991042723333036,0002-9068,NaN,American Jewish Historical Quarterly
954925382034,0004-0843,1923-1245,Arctic
954928472097,0004-2633,NaN,Arnoldia
991042753968002,0005-9315,NaN,Berliner Museen
954927525746,0007-1447,NaN,British medical journal
954925340710,0008-1973,1469-2139,Cambridge law journal
110978976775255,0008-6495,2470-6302,Caribbean Quarterly
3710000000239869,0008-901X,2375-4877,The Centennial Review of Arts & Science


In [24]:
# export the leftover titles and IDs to lists, create empty lists for fuzzy title matching
jstor_fuzzy_record_numbers = leftovers.index.tolist()
jstor_fuzzy_titles = leftovers.TITLE.tolist()

sfx_fuzzy_matches_id = []
sfx_fuzzy_matches_title = []
sirsi_fuzzy_matches_id = []
sirsi_fuzzy_matches_title = []
fuzzy_matches_score = []

In [25]:
sirsi_titles = filtered_issns['title-proper'].tolist()
len(sirsi_titles)

13110

In [26]:
# try to match the leftover titles to the Sirsi records, return only matches that score above 95%
for entry in tqdm(jstor_fuzzy_titles):
    matcher = process.extractOne(entry, sirsi_titles, scorer=fuzz.token_sort_ratio)
    if matcher[1] >= 95:
        sfx_fuzzy_matches_title.append(entry)
        sirsi_fuzzy_matches_title.append(matcher[0])
        fuzzy_matches_score.append(matcher[1])
        sirsi_match_index = sirsi_titles.index(matcher[0])
        sirsi_id = sirsi_record_numbers[sirsi_match_index]
        sirsi_fuzzy_matches_id.append(sirsi_id)
        sfx_match_index = jstor_fuzzy_titles.index(entry)
        sfx_id = jstor_fuzzy_record_numbers[sfx_match_index]
        sfx_fuzzy_matches_id.append(sfx_id)

100%|████████████████████████████████████████| 278/278 [00:58<00:00,  4.72it/s]


In [27]:
# collect the matches in a dataframe
df_fuzzy_matches = pd.DataFrame()
df_fuzzy_matches['sfx_ID'] = sfx_fuzzy_matches_id
df_fuzzy_matches['sirsi_ID'] = sirsi_fuzzy_matches_id
df_fuzzy_matches['sfx_title'] = sfx_fuzzy_matches_title
df_fuzzy_matches['sirsi_title'] = sirsi_fuzzy_matches_title
df_fuzzy_matches['score'] = fuzzy_matches_score
indexed_df_fuzzy_matches = df_fuzzy_matches.set_index(['sfx_ID'])
indexed_df_fuzzy_matches

,sirsi_ID,sfx_title,sirsi_title,score
sfx_ID,,,,
110978976775255,(Sirsi) 2348159,Caribbean Quarterly,Caribbean quarterly,100
954925399506,(Sirsi) 1413042,Family Planning Perspectives,Family planning perspectives,100
954925425184,(Sirsi) 1468413,Mind,Mind,100
991042723365568,(Sirsi) 2266431,Social Science,Social sciences,97
954925450820,(Sirsi) 2022860,University of Pennsylvania Law Review,University of Pennsylvania law review,100
110992357286438,(Sirsi) 2654386,Archiv für Rechts- und Sozialphilosophie,Archiv für Rechts- und Sozialphilosophie,100
110978978554205,(Sirsi) 2198737,Text,Text,100
111039533972032,(Sirsi) 2205809,University of Pennsylvania Law Review and Amer...,University of Pennsylvania law review and Amer...,100
110992357360828,(Sirsi) 3503671,Revue néo-scolastique,Revue néo-scolastique,100


In [28]:
# what's up with entry no. 2320000000017229?
indexed_df_fuzzy_matches.loc[2320000000017229]['sfx_title'], 'vs.', indexed_df_fuzzy_matches.loc[2320000000017229]['sirsi_title']

('Bulletin of the American Association of University Professors (2010-)',
 'vs.',
 'Bulletin of the American Association of University Professors')

In [29]:
# oh, okay, that seems okay. But 991042723365568 and 3710000000613401 appear to be mis-matched, so let's drop them
indexed_df_fuzzy_matches_dropped = indexed_df_fuzzy_matches.drop([991042723365568, 3710000000613401])
indexed_df_fuzzy_matches_dropped

,sirsi_ID,sfx_title,sirsi_title,score
sfx_ID,,,,
110978976775255,(Sirsi) 2348159,Caribbean Quarterly,Caribbean quarterly,100
954925399506,(Sirsi) 1413042,Family Planning Perspectives,Family planning perspectives,100
954925425184,(Sirsi) 1468413,Mind,Mind,100
954925450820,(Sirsi) 2022860,University of Pennsylvania Law Review,University of Pennsylvania law review,100
110992357286438,(Sirsi) 2654386,Archiv für Rechts- und Sozialphilosophie,Archiv für Rechts- und Sozialphilosophie,100
110978978554205,(Sirsi) 2198737,Text,Text,100
111039533972032,(Sirsi) 2205809,University of Pennsylvania Law Review and Amer...,University of Pennsylvania law review and Amer...,100
110992357360828,(Sirsi) 3503671,Revue néo-scolastique,Revue néo-scolastique,100
3710000001079538,(Sirsi) 3384079,Mathematisk Tidsskrift,Mathematisk tidsskrift,100


In [30]:
# Let's merge all three dataframes: the JSTOR list, the cleaned ISSN matches, and
# the fuzzy matches. But first let's rename the JSTOR index column so it's the
# same as the others -- 'sfx_ID'
jstor.index.names = ['sfx_ID']
jstor.head()

,pissn,eissn,TITLE
sfx_ID,,,
991042730655002,0001-026X,2326-9014,AAUP Bulletin
110992357244018,0001-2343,NaN,ARSP: Archiv für Rechts- und Sozialphilosophie...
110985822449210,0001-2890,2168-8052,AV communication review : audio visual communi...
954921333007,0001-4273,1948-0989,Academy of Management Journal
954921333014,0001-4826,1558-7967,Accounting Review


In [31]:
# merge the three dataframes
merge1 = pd.merge(jstor, final_jstor_df, on = 'sfx_ID', how = 'outer')
merge2 = pd.merge(merge1, indexed_df_fuzzy_matches_dropped, on = 'sfx_ID', how = 'outer')
merge2

,pissn,eissn,TITLE,sirsi_ID_x,sirsi_ID_y,sfx_title,sirsi_title,score
sfx_ID,,,,,,,,
991042730655002,0001-026X,2326-9014,AAUP Bulletin,NaN,NaN,NaN,NaN,NaN
110992357244018,0001-2343,NaN,ARSP: Archiv für Rechts- und Sozialphilosophie...,(Sirsi) 2654386,NaN,NaN,NaN,NaN
110985822449210,0001-2890,2168-8052,AV communication review : audio visual communi...,(Sirsi) 2233714,NaN,NaN,NaN,NaN
954921333007,0001-4273,1948-0989,Academy of Management Journal,(Sirsi) 1652669,NaN,NaN,NaN,NaN
954921333014,0001-4826,1558-7967,Accounting Review,(Sirsi) 1652671,NaN,NaN,NaN,NaN
110978976471213,0001-5849,2064-7182,Acta Historica Academiae Scientiarum Hungaricae,(Sirsi) 2915767,NaN,NaN,NaN,NaN
110985822449232,0001-5946,2560-1814,Acta Linguistica Academiae Scientiarum Hungaricae,(Sirsi) 3378318,NaN,NaN,NaN,NaN
954925374811,0001-6241,2296-4339,Acta Musicologica,(Sirsi) 1834281,NaN,NaN,NaN,NaN
110978976471217,0001-6373,1588-2659,Acta Oeconomica,(Sirsi) 2348137,NaN,NaN,NaN,NaN


In [32]:
# Again, let's fill the NaN values with empty spaces, merge, and remove
# Then, replace empty cells with NaN
merge2['sirsi_ID_y'].fillna('', inplace=True)
merge2['sirsi_ID_x'].fillna('', inplace=True)
merge2['sirsi_ID'] = merge2[['sirsi_ID_x', 'sirsi_ID_y']].apply(lambda x: ''.join(x), axis=1)
merge2 = merge2.drop(['sirsi_ID_x', 'sirsi_ID_y'], axis=1)
merge2 = merge2.replace(r'^$', np.nan, regex=True)
merge2

,pissn,eissn,TITLE,sfx_title,sirsi_title,score,sirsi_ID
sfx_ID,,,,,,,
991042730655002,0001-026X,2326-9014,AAUP Bulletin,NaN,NaN,NaN,NaN
110992357244018,0001-2343,NaN,ARSP: Archiv für Rechts- und Sozialphilosophie...,NaN,NaN,NaN,(Sirsi) 2654386
110985822449210,0001-2890,2168-8052,AV communication review : audio visual communi...,NaN,NaN,NaN,(Sirsi) 2233714
954921333007,0001-4273,1948-0989,Academy of Management Journal,NaN,NaN,NaN,(Sirsi) 1652669
954921333014,0001-4826,1558-7967,Accounting Review,NaN,NaN,NaN,(Sirsi) 1652671
110978976471213,0001-5849,2064-7182,Acta Historica Academiae Scientiarum Hungaricae,NaN,NaN,NaN,(Sirsi) 2915767
110985822449232,0001-5946,2560-1814,Acta Linguistica Academiae Scientiarum Hungaricae,NaN,NaN,NaN,(Sirsi) 3378318
954925374811,0001-6241,2296-4339,Acta Musicologica,NaN,NaN,NaN,(Sirsi) 1834281
110978976471217,0001-6373,1588-2659,Acta Oeconomica,NaN,NaN,NaN,(Sirsi) 2348137


In [33]:
# Now it's safe to drop the duplicative SFX title column
merge2 = merge2.drop(['sfx_title'], axis = 1)
merge2

,pissn,eissn,TITLE,sirsi_title,score,sirsi_ID
sfx_ID,,,,,,
991042730655002,0001-026X,2326-9014,AAUP Bulletin,NaN,NaN,NaN
110992357244018,0001-2343,NaN,ARSP: Archiv für Rechts- und Sozialphilosophie...,NaN,NaN,(Sirsi) 2654386
110985822449210,0001-2890,2168-8052,AV communication review : audio visual communi...,NaN,NaN,(Sirsi) 2233714
954921333007,0001-4273,1948-0989,Academy of Management Journal,NaN,NaN,(Sirsi) 1652669
954921333014,0001-4826,1558-7967,Accounting Review,NaN,NaN,(Sirsi) 1652671
110978976471213,0001-5849,2064-7182,Acta Historica Academiae Scientiarum Hungaricae,NaN,NaN,(Sirsi) 2915767
110985822449232,0001-5946,2560-1814,Acta Linguistica Academiae Scientiarum Hungaricae,NaN,NaN,(Sirsi) 3378318
954925374811,0001-6241,2296-4339,Acta Musicologica,NaN,NaN,(Sirsi) 1834281
110978976471217,0001-6373,1588-2659,Acta Oeconomica,NaN,NaN,(Sirsi) 2348137


In [34]:
# and finally, rearrange
merge2 = merge2[['pissn', 'eissn', 'TITLE', 'sirsi_ID', 'sirsi_title', 'score']]
merge2

,pissn,eissn,TITLE,sirsi_ID,sirsi_title,score
sfx_ID,,,,,,
991042730655002,0001-026X,2326-9014,AAUP Bulletin,NaN,NaN,NaN
110992357244018,0001-2343,NaN,ARSP: Archiv für Rechts- und Sozialphilosophie...,(Sirsi) 2654386,NaN,NaN
110985822449210,0001-2890,2168-8052,AV communication review : audio visual communi...,(Sirsi) 2233714,NaN,NaN
954921333007,0001-4273,1948-0989,Academy of Management Journal,(Sirsi) 1652669,NaN,NaN
954921333014,0001-4826,1558-7967,Accounting Review,(Sirsi) 1652671,NaN,NaN
110978976471213,0001-5849,2064-7182,Acta Historica Academiae Scientiarum Hungaricae,(Sirsi) 2915767,NaN,NaN
110985822449232,0001-5946,2560-1814,Acta Linguistica Academiae Scientiarum Hungaricae,(Sirsi) 3378318,NaN,NaN
954925374811,0001-6241,2296-4339,Acta Musicologica,(Sirsi) 1834281,NaN,NaN
110978976471217,0001-6373,1588-2659,Acta Oeconomica,(Sirsi) 2348137,NaN,NaN
